<a href="https://colab.research.google.com/github/maidacundo/test_referees_optimization/blob/main/test_ortools3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install ortools
from ortools.sat.python import cp_model
import multiprocessing

In [23]:
def make_day(num_teams, day):
    # using circle algorithm, https://en.wikipedia.org/wiki/Round-robin_tournament#Scheduling_algorithm
    assert not num_teams % 2, "Number of teams must be even!"
    # generate list of teams
    lst = list(range(num_teams))
    # rotate
    day %= (num_teams - 1)  # clip to 0 .. num_teams - 2
    if day:                 # if day == 0, no rotation is needed (and using -0 as list index will cause problems)
        lst = lst[:1] + lst[-day:] + lst[1:-day]
    # pair off - zip the first half against the second half reversed
    half = num_teams // 2
    return list(zip(lst[:half], lst[half:][::-1]))

def make_schedule(num_teams):
    """
    Produce a double round-robin schedule
    """
    # number of teams must be even
    if num_teams % 2:
        num_teams += 1  # add a dummy team for padding

    # build first round-robin
    schedule = [make_day(num_teams, day) for day in range(num_teams - 1)]
    # generate second round-robin by swapping home,away teams
    swapped = [[(away, home) for home, away in day] for day in schedule]

    return schedule + swapped

In [24]:
def make_prediction(num_referee, num_teams, calendar, referee_history):

  class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
      """Print intermediate solutions."""

      def __init__(self, variables):
          cp_model.CpSolverSolutionCallback.__init__(self)
          self.__variables = variables
          self.__solution_count = 0

      def on_solution_callback(self):
          self.__solution_count += 1
          print(f'trovata soluzione num {self.__solution_count}')
          print(f'Differenza totale per partite arbitrate: {self.Value(total_diff_per_referre_to_minimize)}')
          print(f'Differenza totale per squadre arbitrate: {self.Value(team_diff_per_referre_to_minimize)}')

      def solution_count(self):
          return self.__solution_count

  model = cp_model.CpModel()

  x = []
  for i in range(num_referee):
      r = []
      for day in range(num_days):
          num_matches = len(calendar[day])
          d = []
          for j in range(num_matches):
              d.append(model.NewIntVar(0, 1, 'referee{}_day{}_match{}'.format(i, day, j)))
          r.append(d)
      x.append(r)

  # Ogni arbitro è assegnato al max ad una partita per giorno
  for i in range(num_referee):
      for d in range(num_days):
          model.AddAtMostOne(x[i][d][j] for j in range(num_matches))

  # Ogni partita ha 3 arbitri
  for d in range(num_days):
      for j in range(num_matches):
          model.Add(sum([x[i][d][j] for i in range(num_referee)]) == min_referee_per_match)

  # calcolo delle squadre arbitrate da ogni singolo arbitro
  referee_matches_per_team = []
  for i in range(num_referee):
    teams_per_referee = []
    for s in range(num_teams):
      matches_per_team = []
      for d in range(num_days):
        for m in calendar[d]:
          if s in m:
            j = calendar[d].index(m)
            matches_per_team.append(x[i][d][j])
      try:
        previous_matches_number = referee_history[i][s]
      except IndexError:
        print('errore')
        previous_matches_number = 0
      teams_per_referee.append(sum(matches_per_team) + previous_matches_number)
    referee_matches_per_team.append(teams_per_referee)

  team_diff = []
  all_variables = []

  for i in range(len(referee_matches_per_team)):
    t = []
    for s in range(num_teams):
      c = []
      for j in range(len(referee_matches_per_team)):
        if i != j:
          diff_var = model.NewIntVar(0, 100, f'matches_diff_{i}_{j}_team_{s}') # todo controllare meglio up e lb!
          abs_diff = model.AddAbsEquality(diff_var, (referee_matches_per_team[i][s] - referee_matches_per_team[j][s]))
          c.append(diff_var)
          all_variables.append(diff_var)
      
      t.append(sum(c))
    team_diff.append(sum(t))

  # aggiungo una decision stategy (euristica) per migliorare le performance
  model.AddDecisionStrategy(all_variables, cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_MIN_VALUE)

  team_diff_per_referre_to_minimize = sum(team_diff)

  # calcolo dei match totali arbitrati da ogni singolo arbitro
  total_matches_per_referee = []

  for i in range(num_referee):
      total_matches = 0
      for d in range(num_days):
          total_matches += sum(x[i][d])
      total_matches_per_referee.append(total_matches)

  matches_differences = []
  all_variables = []

  for i in range(len(total_matches_per_referee)):
      c = []
      for j in range(len(total_matches_per_referee)):
          if i != j:
              diff_var = model.NewIntVar(0, 100, f'matches_diff_{i}_{j}')
              abs_diff = model.AddAbsEquality(diff_var, (total_matches_per_referee[i] - total_matches_per_referee[j]))
              c.append(diff_var)
              all_variables.append(diff_var)
      matches_differences.append(c)

  # aggiungo una decision stategy (euristica) per migliorare le performance
  model.AddDecisionStrategy(all_variables, cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_LOWER_HALF)

  total_differences_per_referee = []
  for i in range(len(matches_differences)):
      total_differences_per_referee.append(sum(matches_differences[i]))
    
  total_diff_per_referre_to_minimize = sum(total_differences_per_referee)

  # lista contenente tutti i valori da minimizzare
  values_to_minimize = []
  values_to_minimize.append(team_diff_per_referre_to_minimize * 0.7)
  values_to_minimize.append(total_diff_per_referre_to_minimize * 0.3)

  model.Minimize(sum(values_to_minimize))
  # model.Minimize(team_diff_per_referre_to_minimize)

  print(model.ModelStats())
  solver = cp_model.CpSolver()
  solution_printer = VarArraySolutionPrinter(x)

  #solver.parameters.enumerate_all_solutions = True
  solver.parameters.num_search_workers = multiprocessing.cpu_count()

  solver.parameters.max_time_in_seconds = 480

  status = solver.Solve(model, solution_printer)

  solution = []

  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:

      print('POSSIBILE SOLUZIONE')
      match_per_referee = [0] * num_referee

      for i in range(num_referee):
          solution.append([])
          num_matches = len(calendar[d])
          for d in range(num_days):
              for j in range(num_matches):
                  if solver.BooleanValue(x[i][d][j]):
                      match_per_referee[i] += 1
                      print(
                          f'Referee {i} assigned to match {calendar[d][j]}')
                      solution[i] = calendar[d][j]
      print()
      
      print(f'Differenza totale per partite arbitrate: {solver.Value(total_diff_per_referre_to_minimize)}')
      print(f'Differenza totale per squadre arbitrate: {solver.Value(team_diff_per_referre_to_minimize)}')

      print()
      scores_team_diff = []
      print('TOTALI PARTITE PER SQUADRA')
      for i in range(num_referee):
        print(f'Referee {i}: {solver.Value(team_diff[i])}')
        scores_team_diff.append(solver.Value(team_diff[i]))

  return solution

In [25]:
num_referee = 80
num_teams = 20

calendar = make_schedule(num_teams)

num_days = len(calendar)
min_referee_per_match = 3

referee_history = []
for i in range(num_referee):
  referee_history.append([])
  for s in range(num_teams):
    referee_history[i].append(0)

giornata = []
print(calendar)

for i in range(len(calendar)):
  giornata = calendar[i:i+1]
  print(giornata)
  num_days = len(giornata)
  solution = make_prediction(num_referee, num_teams, giornata, referee_history)
  for i in range(num_referee):
    for j in solution[i]:
      referee_history[i][j] += 1

Streaming output truncated to the last 5000 lines.
Referee 6: 510
Referee 7: 608
Referee 8: 506
Referee 9: 634
Referee 10: 708
Referee 11: 632
Referee 12: 506
Referee 13: 632
Referee 14: 404
Referee 15: 404
Referee 16: 514
Referee 17: 408
Referee 18: 746
Referee 19: 608
Referee 20: 720
Referee 21: 718
Referee 22: 842
Referee 23: 746
Referee 24: 618
Referee 25: 766
Referee 26: 638
Referee 27: 512
Referee 28: 636
Referee 29: 404
Referee 30: 504
Referee 31: 508
Referee 32: 510
Referee 33: 402
Referee 34: 406
Referee 35: 532
Referee 36: 402
Referee 37: 300
Referee 38: 406
Referee 39: 400
Referee 40: 400
Referee 41: 634
Referee 42: 506
Referee 43: 404
Referee 44: 300
Referee 45: 300
Referee 46: 402
Referee 47: 300
Referee 48: 300
Referee 49: 400
Referee 50: 502
Referee 51: 400
Referee 52: 404
Referee 53: 400
Referee 54: 614
Referee 55: 512
Referee 56: 300
Referee 57: 300
Referee 58: 404
Referee 59: 638
Referee 60: 300
Referee 61: 300
Referee 62: 508
Referee 63: 300
Referee 64: 736
Referee 6

In [26]:
import numpy as np

# 0.731875
# 0.474375
# 0.380625
np.var(referee_history)

0.380625

In [27]:
referee_history

[[1, 1, 1, 1, 1, 1, 3, 1, 2, 1, 1, 2, 1, 2, 2, 3, 2, 3, 2, 1],
 [3, 1, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2],
 [1, 1, 3, 1, 1, 1, 2, 2, 2, 3, 2, 2, 1, 1, 3, 2, 1, 1, 2, 2],
 [2, 2, 2, 2, 3, 1, 1, 1, 2, 1, 2, 3, 1, 4, 1, 2, 1, 1, 2, 4],
 [1, 2, 2, 1, 1, 1, 1, 3, 2, 2, 2, 3, 1, 1, 1, 2, 1, 4, 2, 1],
 [1, 2, 1, 1, 2, 2, 1, 3, 1, 2, 1, 2, 2, 1, 1, 1, 2, 3, 1, 2],
 [1, 2, 2, 2, 2, 1, 2, 3, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2],
 [2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1],
 [1, 2, 1, 2, 1, 2, 3, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2],
 [2, 2, 2, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2, 1, 3, 2],
 [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2],
 [1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 3, 2, 3, 2, 1, 2, 1, 1],
 [2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 3, 1, 1, 1],
 [2, 1, 1, 1, 1, 2, 1, 2, 2, 3, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1],
 [2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3],
 [1, 2, 4, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 3, 1,